<h3>Hockin-Mann model from Owen et al</h3>

Owen MJ, Wright JR, Tuddenham EGD, King JR, Goodall AH, Dunster JL. Mathematical models of coagulation-are we there yet? J Thromb Haemost. 2024 Jun;22(6):1689-1703. doi: 10.1016/j.jtha.2024.03.009.

This analysis uses a JupyterLab session as a wrapper to import the python module hockin_modified.py which has reactions added for thrombin inhibition by heparin cofactor II, $\alpha$-1-protease, and $\alpha$-2-macroglobulin

In [ ]:
import numpy as np
import scipy.integrate
import pandas
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams, cycler  #needed to make color gradients
%matplotlib inline

In [ ]:
print('numerical python version: ',np.__version__)
print('Pandas version: ',pandas.__version__)
print('Matplotlib version: ', matplotlib.__version__)

In [ ]:
import hockin_modified

In [ ]:
y = hockin_modified.setIC();
k = hockin_modified.getRates();

In [ ]:
normal = pandas.DataFrame(hockin_modified.getThr(k,y,2400), index=['seconds','thrombin']).transpose()

In [ ]:
# display(normal)

In [ ]:
#set up dictionary for the data
HemA_data = {}
# factor VIII concentration is y[14]
# antithrombin concentration is y[28]
for i in [0.0, 0.03, 0.09, 0.15, 0.30, 1.0]:
    y = hockin_modified.setIC()
    y[14] = 0.0
    y[28] = i*y[28]
    sheetname = "AT " + str(round(i*100)) + "%"
    df = pandas.DataFrame(hockin_modified.getThr(k,y,2400), index=['seconds','thrombin']).transpose()
    HemA_data.update({sheetname: df})

In [ ]:
df_sheet = pandas.DataFrame(HemA_data.keys())
display(df_sheet)

In [ ]:
fig4, ax = plt.subplots(nrows=1, ncols=2, figsize=(13,6))
for i in range(2):
    ax[i].tick_params(which='both', direction='in', 
                      length=4, width=2, colors='k', 
                      grid_color='k', grid_alpha=1, labelsize=20)
# 
# plot the results of the thrombin generation model
# convert time to minutes
# convert molar to nM
#
ax[0].plot(normal.loc[:,'seconds']/60, 
           normal.loc[:,'thrombin']*1e9, 
           'g-', linewidth=3, label='normal')
#
ax[1].plot(normal.iloc[:,0]/60,
           scipy.integrate.cumulative_trapezoid(normal.loc[:,'thrombin']*1e9,
                                                normal.loc[:,'seconds']/60, 
                                                initial=0), 'g-', linewidth=3)
# 
# plot the results of the thrombin generation model in hemophilia with varied antithrombin
#
cmap = plt.cm.seismic
rcParams['axes.prop_cycle'] = cycler(color=cmap(np.linspace(0, 1, len(df_sheet))))
# 
for i in range(len(df_sheet)):
    x = HemA_data[df_sheet.iloc[i,0]].loc[:,'seconds']/60
    y = HemA_data[df_sheet.iloc[i,0]].loc[:,'thrombin']*1e9
    y1 = scipy.integrate.cumulative_trapezoid(y, x, initial=0)
    ax[0].plot(x, y, linewidth=3, label=df_sheet.iloc[i])
    ax[1].plot(x, y1, linewidth=3)
#
ax[0].set_yticks(np.arange(0, 70, step=10))
ax[0].set_ylabel("[thrombin] (nM)", fontsize=20)
ax[1].set_ylabel("integrated thrombin", fontsize=20)
for i in range(2):
    ax[i].set_xlabel("time (minutes)", fontsize=20)
# 
ax[0].legend(loc='upper right', fontsize=14, labelspacing=0.2, frameon=False)
#
identifiers = ['A', 'B']
for i in range(2):
    x_min, x_max = ax[i].get_xlim()
    y_min, y_max = ax[i].get_ylim()
    ax[i].text(x_min-0.15*(x_max-x_min), 
               y_max+(0.05*(y_max-y_min)), 
               identifiers[i], 
               fontsize=36)
#
fig4.tight_layout(pad=2.0)

<h3>"Normal" range of thrombin generation</h3>

I am not sure how to put the increased thrombin generation in a context. Maybe one way is to generate a range of thrombin generation between when all the inhibitors are at the highest values of their normal range and when all the inhibitors are at the lowest values of their normal range.

* AT 86% - 128%
* HC 70% - 130%
* a1PI 53% - 133%
* a2M 70% - 155%

In [ ]:
t_start = 0
t_end = 2400
t_interval = 201
t_points = np.linspace(t_start, t_end, t_interval)
#
# antithrombin concentration is y[28]
# heparin cofactor II is y[34]
# alpha-1-protease inhibitor is y[35]
# alpha-2-macroglobulin is y[36]
#
y = hockin_modified.setIC()
k = hockin_modified.getRates()
y[28] = 0.86*y[28]
y[34] = 0.7*y[34]
y[35] = 0.53*y[35]
y[36] = 0.7*y[36]
high_low_data.update({"low": scipy.integrate.solve_ivp(hockin_modified.ODE, 
                                                       (t_start,t_end), y, 
                                                       args=(k, ), 
                                                       t_eval=t_points, 
                                                       method='BDF', 
                                                       rtol=1e-4,atol=1e-16)})
#
y = hockin_modified.setIC()
k = hockin_modified.getRates()
y[28] = 1.28*y[28]
y[34] = 1.3*y[34]
y[35] = 1.33*y[35]
y[36] = 1.55*y[36]
high_low_data.update({"high": scipy.integrate.solve_ivp(hockin_modified.ODE, 
                                                        (t_start,t_end), y, 
                                                        args=(k, ), 
                                                        t_eval=t_points, 
                                                        method='BDF', 
                                                        rtol=1e-4,atol=1e-16)})
#

In [ ]:
df_low.to_csv('low.csv', 
              sep=',',
              index=False,
              header=True)
#
df_high.to_csv('high.csv', 
               sep=',',
               index=False,
               header=True)

In [ ]:
fig5, ax = plt.subplots(nrows=1, ncols=2, figsize=(13,6))
for i in range(2):
    ax[i].tick_params(which='both', direction='in', 
                      length=4, width=2, colors='k', 
                      grid_color='k', grid_alpha=1, labelsize=20)
# 
# plot the results of the thrombin generation model
# convert time to minutes
# convert molar to nM
#
ax[0].plot(normal.loc[:,'seconds']/60, 
           normal.loc[:,'thrombin']*1e9, 
           'g-', linewidth=3, label='normal')
#
ax[1].plot(normal.iloc[:,0]/60,
           scipy.integrate.cumulative_trapezoid(normal.loc[:,'thrombin']*1e9,
                                                normal.loc[:,'seconds']/60, 
                                                initial=0), 'g-', linewidth=3)
# 
# plot thrombin generation within the inhibitors reference range extremes
# 
ax[0].fill_between(high_low_data['low'].t/60, 
                   high_low_data['low'].y[6]*1e9, 
                   high_low_data['high'].y[6]*1e9, 
                   color='g', alpha=0.3)
# 
y1i = scipy.integrate.cumulative_trapezoid(high_low_data['low'].y[6]*1e9, 
                                           high_low_data['low'].t/60, initial=0)
y2i = scipy.integrate.cumulative_trapezoid(high_low_data['high'].y[6]*1e9, 
                                           high_low_data['high'].t/60, initial=0)
ax[1].fill_between(high_low_data['low'].t/60, 
                   y1i, 
                   y2i, 
                   color='g', alpha=0.3)
# 
# plot the results of the thrombin generation model in hemophilia with varied antithrombin
#
cmap = plt.cm.seismic
rcParams['axes.prop_cycle'] = cycler(color=cmap(np.linspace(0, 1, len(df_sheet))))
# 
for i in range(len(df_sheet)):
    x = HemA_data[df_sheet.iloc[i,0]].loc[:,'seconds']/60
    y = HemA_data[df_sheet.iloc[i,0]].loc[:,'thrombin']*1e9
    y1 = scipy.integrate.cumulative_trapezoid(y, x, initial=0)
    ax[0].plot(x, y, linewidth=3, label=df_sheet.iloc[i])
    ax[1].plot(x, y1, linewidth=3)
#
ax[0].set_yticks(np.arange(0, 90, step=10))
ax[0].set_ylabel("[thrombin] (nM)", fontsize=20)
ax[1].set_ylabel("integrated thrombin", fontsize=20)
for i in range(2):
    ax[i].set_xlabel("time (minutes)", fontsize=20)
# 
ax[0].legend(loc='upper right', fontsize=14, labelspacing=0.2, frameon=False)
#
identifiers = ['A', 'B']
for i in range(2):
    x_min, x_max = ax[i].get_xlim()
    y_min, y_max = ax[i].get_ylim()
    ax[i].text(x_min-0.15*(x_max-x_min), 
               y_max+(0.05*(y_max-y_min)), 
               identifiers[i], 
               fontsize=36)
#
fig5.tight_layout(pad=2.0)

In [ ]:
# fig5.savefig('Figure_4.svg', bbox_inches='tight')